In [1]:
# -*- coding: utf-8 -*-
"""deep_distance_estimatorPooja.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1wD7Z_pun1_dkbuxyEwfZxy14pO5Zvgoa

Code from Rienke on Github, from repository single_cell_segmentation
- only edited to try and work with data from NeuroToolBox Gong Lab

Deep Distance Estimator is trained to learn EDT.
EDT is the euclidean distance transform and emasures the distance between
two pixels


a deep distance estimator is trained to learn EDT. Since the Euclidean distance assumes a continuous instead of a Boolean value, pixels that are close to cell boundary have smaller values. Specifically consider the regions with blurred boundaries in the original images.

MUST use conda environment
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
#Use this when running in google colab
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
import numpy as np
import pandas as pd
import sys
# import cv2
from skimage.io import imread, imread_collection
from scipy import signal
import scipy.misc
# from scipy.misc import imread
from scipy.ndimage import filters,distance_transform_edt
from matplotlib import pyplot as plt
import imageio # use im = imageio.imread(_____)
import os
from os import listdir

# import tensorflow as tf
# print(tf.version.VERSION)

from keras.models import Model
from keras import optimizers
from keras.layers import *

Using TensorFlow backend.


In [4]:
dir_path='/Users/poojap/Documents/ProteinSensorResearch/single_cell_segmentation-master'
main_path=dir_path+'/NMuMg_phase_contrast/training_data'
path_train_input = main_path+'/distance_transform/train/Img'
# no testing file
path_test_input= main_path+'/distance_transform/testImg'
path_train_gt = main_path+'/distance_transform/train/Bwdist'
#no testing file
path_test_gt=main_path+'/distance_transform/testBWdist'

# what is predict?

path_predict= '/content/gdrive/MyDrive/ImageSegmentationFiles/MembraneImages'
# path_predict = '/content/gdrive/MyDrive/single_cell_segmentation-master/NMuMg_phase_contrast/training_data/distance_transform/testImg/crop_phasexy18c2cr1s8.tif'
path_output= '/content/gdrive/MyDrive/single_cell_segmentation-master/output/deepDistanceEstimator'

# This pathway is still wrong
path_frcnn_predict=dir_path+'/frcnn_predict'

nb_epoch =50
batch_size = 10
weight_file= dir_path + '/nmumg_reg_Pooja_5epochs.hdf5'

In [5]:
#As this is a regression problem, train_label should be also normalized.
def prep_data(path_img,path_gt):
    data = []
    label = []

    img_list = sorted(listdir(path_img))
    gt_list=sorted(listdir(path_gt))

    for i in range(1, len(img_list)):
        # imageio.imread is not working .. try skimage.io import imread_collection
        newimg = (path_train_input + '/' + img_list[i])
        print(img_list[i])
        img = np.array(imageio.imread(newimg),dtype=np.float64)
        #img = np.array(imread(path_img +'/'+ img_list[i]))
        #print img.shape
        img=(img-np.amin(img))*1.0/(np.amax(img)-np.amin(img))#img*1.0 transform array to double
        img=img*1.0/np.median(img)
        img_h=img.shape[0]        
        img_w=img.shape[1]        
        img=np.reshape(img,(img_h,img_w,1))         
        data.append(img)  
        
        gt =np.array(imageio.imread(path_gt + '/'+ gt_list[i]))
        print(gt_list[i])
#---------------with or without normalization-----------------------        
        if np.count_nonzero(gt)!=0:
            nonzero_gt=gt[gt>0]
            gt=gt*1.0/np.median(nonzero_gt)

        gt=np.reshape(gt,(img_h,img_w,1))
        label.append(gt)
        
#         i+=1
    data=np.array(data)
    label=np.array(label) 
    print(data.shape, label.shape)
    return data, label     

In [6]:
img_list = sorted(listdir(path_train_input))
# gt_list=sorted(listdir(path_gt))

# while (i <(len(path_train_input))):
for i in range(1, len(img_list)):
    newimg = (path_train_input + '/' + img_list[i])
    imageio.imread(newimg)


In [7]:
def prep_prediction_data(path_img):
    data = []
    img_list = sorted(listdir(path_img))
    i=0
    while (i<len(img_list)):
        img = np.array(imageio.imread(path_img +'/'+ img_list[i]),dtype=np.float64)
        img=(img-np.amin(img))*1.0/(np.amax(img)-np.amin(img))#img*1.0 transform array to double
        img=img*1.0/np.median(img)
        img_h=img.shape[0]        
        img_w=img.shape[1]
        img=np.reshape(img,(img_h,img_w,1))
        data.append(img)
        i+=1
    data=np.array(data)
    return data

In [8]:
def conv_block(input_tensor, kernel, filters):
    x=Conv2D(filters, (kernel, kernel), padding='same')(input_tensor)
    x=BatchNormalization()(x)
    x=Activation('relu')(x)
    return x


In [9]:
kernel = 3
#------------encoder layers--------------------------------
#Before ...keras.Input(None, None, 1)
inputs = Input((None,None, 1))
print(inputs)
conv1=conv_block(inputs,kernel,filters=64)
conv1=conv_block(conv1,kernel,filters=64)
pool1=MaxPooling2D()(conv1)
    
conv2=conv_block(pool1,kernel,filters=128)
conv2=conv_block(conv2,kernel,filters=128)
pool2=MaxPooling2D()(conv2)

conv3=conv_block(pool2,kernel,filters=256)
conv3=conv_block(conv3,kernel,filters=256)
conv3=conv_block(conv3,kernel,filters=256)
pool3=MaxPooling2D()(conv3)

conv4=conv_block(pool3,kernel,filters=512)
conv4=conv_block(conv4,kernel,filters=512)
conv4=conv_block(conv4,kernel,filters=512)
pool4=MaxPooling2D()(conv4)
 
conv5=conv_block(pool4,kernel,filters=512)
conv5=conv_block(conv5,kernel,filters=512)
conv5=conv_block(conv5,kernel,filters=512)
pool5=MaxPooling2D()(conv5)


#--------------------decoder layers--------------------------

up6=UpSampling2D()(pool5)
conv6=conv_block(up6,kernel,filters=512)
conv6=conv_block(conv6,kernel,filters=512)
conv6=conv_block(conv6,kernel,filters=512)

up7=UpSampling2D()(conv6)
conv7=conv_block(up7,kernel,filters=512)
conv7=conv_block(conv7,kernel,filters=512)
conv7=conv_block(conv7,kernel,filters=512)
    
up8=UpSampling2D()(conv7)
conv8=conv_block(up8,kernel,filters=256)
conv8=conv_block(conv8,kernel,filters=256)
conv8=conv_block(conv8,kernel,filters=256)

up9=UpSampling2D()(conv8)
conv9=conv_block(up9,kernel,filters=128)
conv9=conv_block(conv9,kernel,filters=128)

up10=UpSampling2D()(conv9)
conv10=conv_block(up10,kernel,filters=64)

conv11=conv_block(conv10,kernel=1,filters=1)    
outputs=Activation('relu')(conv11)

autoencoder=Model(inputs=[inputs], outputs=[outputs])

#autoencoder.summary()

autoencoder.compile(loss="mean_squared_error", optimizer='adam', metrics=['mae','acc'])
print ('Compiled: OK')

Tensor("input_1:0", shape=(None, None, None, 1), dtype=float32)
Compiled: OK


In [11]:
train_data, train_label = prep_data(path_train_input,path_train_gt)

# Each data matches with the label - checked with print statements

crop_phasexy01c2cr1s0.tif
bwdist_phasexy01c2cr1s0.tif
crop_phasexy01c2cr1s1.tif
bwdist_phasexy01c2cr1s1.tif
crop_phasexy01c2cr1s2.tif
bwdist_phasexy01c2cr1s2.tif
crop_phasexy01c2cr1s3.tif
bwdist_phasexy01c2cr1s3.tif
crop_phasexy01c2cr1s4.tif
bwdist_phasexy01c2cr1s4.tif
crop_phasexy01c2cr1s5.tif
bwdist_phasexy01c2cr1s5.tif
crop_phasexy01c2cr1s6.tif
bwdist_phasexy01c2cr1s6.tif
crop_phasexy01c2cr1s7.tif
bwdist_phasexy01c2cr1s7.tif
crop_phasexy01c2cr1s8.tif
bwdist_phasexy01c2cr1s8.tif
crop_phasexy01c2cr1s9.tif
bwdist_phasexy01c2cr1s9.tif
crop_phasexy01c2cr2s0.tif
bwdist_phasexy01c2cr2s0.tif
crop_phasexy01c2cr2s1.tif
bwdist_phasexy01c2cr2s1.tif
crop_phasexy01c2cr2s2.tif
bwdist_phasexy01c2cr2s2.tif
crop_phasexy01c2cr2s3.tif
bwdist_phasexy01c2cr2s3.tif
crop_phasexy02c2cr1s0.tif
bwdist_phasexy02c2cr1s0.tif
crop_phasexy02c2cr1s1.tif
bwdist_phasexy02c2cr1s1.tif
crop_phasexy02c2cr1s10.tif
bwdist_phasexy02c2cr1s10.tif
crop_phasexy02c2cr1s11.tif
bwdist_phasexy02c2cr1s11.tif
crop_phasexy02c2cr1s12.t

In [ ]:
#---------------------train-------------------------
history=autoencoder.fit(train_data, train_label, batch_size=batch_size, epochs=nb_epoch, verbose=1)
autoencoder.save_weights(weight_file)

Epoch 1/50
80/80 [==============================] - 560s 7s/step - loss: 0.7251 - mae: 0.5289 - acc: 0.4685
Epoch 2/50
80/80 [==============================] - 851s 11s/step - loss: 0.5376 - mae: 0.4531 - acc: 0.4161
Epoch 3/50
80/80 [==============================] - 3188s 40s/step - loss: 0.5339 - mae: 0.4630 - acc: 0.3836
Epoch 4/50
80/80 [==============================] - 564s 7s/step - loss: 0.4832 - mae: 0.4266 - acc: 0.4307
Epoch 5/50
80/80 [==============================] - 559s 7s/step - loss: 0.3864 - mae: 0.3683 - acc: 0.4615
Epoch 6/50
80/80 [==============================] - 1574s 20s/step - loss: 0.3536 - mae: 0.3463 - acc: 0.4705
Epoch 7/50
80/80 [==============================] - 566s 7s/step - loss: 0.2830 - mae: 0.3031 - acc: 0.5087
Epoch 8/50
80/80 [==============================] - 562s 7s/step - loss: 0.2575 - mae: 0.2857 - acc: 0.5108
Epoch 9/50
80/80 [==============================] - 552s 7s/step - loss: 0.2306 - mae: 0.2622 - acc: 0.5158
Epoch 10/50
80/80 [====

In [ ]:
from PIL import Image
from scipy import ndimage
#-------------------predict----------------------------
autoencoder.load_weights(weight_file)
# put htis into the test path same as predict
predict_data=prep_prediction_data(path_predict)
output = autoencoder.predict(predict_data, verbose=0)
print(type(output))
print(output.shape)

#output_label=np.argmax(output[0],axis=-1)
im=output[10][:,:,0]   
np.save(path_output+'/reg_out.npy',im)
data=np.load(path_output+'/reg_out.npy')
print(im.shape)

#save image to 8 bit

imageio.imwrite(path_output+'/reg_out_8bit.png',im)
#save image to the exat value

# img=scipy.misc.toimage(im,high=np.max(im),low=np.min(im),mode='F')
img = Image.fromarray(im, mode = 'F')
img.save(path_output+'/reg_out_exact.tif')
img.save(path_frcnn_predict+'/reg_out_exact.tif')
plt.pyplot.imshow(im
plt.pyplot.axis('off')
plt.pyplot.savefig(path_output+'/reg_out_300dpi.tif',bbox_inches='tight',format='tif',dpi=300)
plt.pyplot.show()
